In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crop-price-final/crop_price_final.csv


In [3]:
df= pd.read_csv('/kaggle/input/crop-price-final/crop_price_final.csv')

In [4]:
df = df.dropna()

In [5]:
df2 = df.copy()

In [6]:
df2.shape

(3036193, 9)

In [7]:
dt = df2['arrival_date'][28]
ar = dt.split('-')
ar

['05', '02', '2022']

In [8]:
Dict = {1:"January", 2:"February", 3:"March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}

In [9]:
month = []
year = []
for rr in df2['arrival_date']:
  str = rr
  str2=str.split('-')
  month.append(Dict[int(str2[1])])
  year.append(int(str2[2]))

In [10]:
len(month)

3036193

In [11]:
df2['month'] = month
df2['year'] = year

In [12]:
day_of_week = []
df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%d-%m-%Y')
for rr in df2['arrival_date']:
  str = rr
  df = pd.Timestamp(rr)
  day = df.dayofweek
  day_of_week.append(day)
  
len(day_of_week)

3036193

In [13]:
df2['day'] = day_of_week

In [14]:
df2.head()

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price,month,year,day
0,Goa,North Goa,Mapusa,Arecanut(Betelnut/Supari),Other,01-01-2022,38600.0,46600.0,42600.0,January,2022,5
1,Goa,North Goa,Mapusa,Arecanut(Betelnut/Supari),Other,03-01-2022,38600.0,46600.0,42600.0,January,2022,1
2,Goa,North Goa,Mapusa,Arecanut(Betelnut/Supari),Other,04-01-2022,38600.0,46300.0,42450.0,January,2022,4
3,Goa,North Goa,Mapusa,Arecanut(Betelnut/Supari),Other,05-01-2022,38600.0,46300.0,42450.0,January,2022,6
4,Goa,North Goa,Mapusa,Arecanut(Betelnut/Supari),Other,08-01-2022,38600.0,46300.0,42450.0,January,2022,0


In [15]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoders
label_encoders = {}

# List of columns to encode
columns_to_encode = ['state', 'district', 'market', 'commodity','variety', 'month']

# Encode labels for each column and save the encoder along with mappings
for column in columns_to_encode:
    label_encoder = LabelEncoder()
    df2[column] = label_encoder.fit_transform(df2[column])
    label_encoders[column] = label_encoder

In [16]:
df2 = df2.drop('arrival_date', axis=1)
df2 = df2.drop('min_price', axis=1)
df2 = df2.drop('max_price', axis=1)

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3036193 entries, 0 to 3041217
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   state        int64  
 1   district     int64  
 2   market       int64  
 3   commodity    int64  
 4   variety      int64  
 5   modal_price  float64
 6   month        int64  
 7   year         int64  
 8   day          int64  
dtypes: float64(1), int64(8)
memory usage: 231.6 MB


In [18]:
from sklearn.model_selection import train_test_split
X = df2.drop('modal_price', axis=1)
y = df2['modal_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train.head()

,state,district,market,commodity,variety,month,year,day
2416875,29,335,1373,35,255,0,2023,3
1271823,23,10,1239,29,255,3,2023,3
1278377,23,162,966,29,255,5,2023,2
600224,7,39,2392,13,278,3,2024,3
541421,15,510,128,13,137,7,2022,3


In [20]:

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 43651233.65088042
R-squared Score: 0.08299705744258068


In [21]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("testing")
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

testing
Mean Squared Error: 6196285.399058818
R-squared Score: 0.8698315839294062


In [22]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("testing")
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")
dtr = model



testing
Mean Squared Error: 3798329.076888303
R-squared Score: 0.9202066322302502


In [23]:
model = RandomForestRegressor(max_depth = 10, random_state = 0, n_estimators = 100)
model.fit(X_train, y_train)
# Predict on the testing set
y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("testing")
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

testing
Mean Squared Error: 289545623.5675712
R-squared Score: -5.08262737634519


In [61]:
pred = dtr.predict([[29,335,1373,35,255,0,2023,3]])
pred

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([3300.])

In [62]:
import pickle

In [63]:
filename = 'price_prediction_trained2.sav'
pickle.dump(dtr, open(filename, 'wb'))


In [64]:
label_encoders

{'state': LabelEncoder(),
 'district': LabelEncoder(),
 'market': LabelEncoder(),
 'commodity': LabelEncoder(),
 'variety': LabelEncoder(),
 'month': LabelEncoder()}

In [65]:
filename2 = 'preprocessor2.sav'
pickle.dump(label_encoders, open(filename2, 'wb'))

In [70]:
def predict_modal_price(state, district, market, commodity, variety, month, year, day):
    
    # Transform categorical features using label encoders
    state = label_encoders['state'].transform([state])[0]
    district = label_encoders['district'].transform([district])[0]
    market = label_encoders['market'].transform([market])[0]
    commodity = label_encoders['commodity'].transform([commodity])[0]
    variety = label_encoders['variety'].transform([variety])[0]
    month = label_encoders['month'].transform([month])[0]

    # Make predictions
    prediction = model.predict([[state, district, market, commodity, variety, month, year, day]])
    
    return prediction[0]

# Example usage:
state = 'Goa'
district = 'North Goa'
market = 'Mapusa'
commodity = 'Arecanut(Betelnut/Supari)'
variety = 'Other'
month = 'January'
year = 2022
day = 3

predicted_price = predict_modal_price(state, district, market, commodity, variety, month, year, day)
print("Predicted Modal Price:", predicted_price)

Predicted Modal Price: 42300.0


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
